# Exception Handling

The functions described in this chapter will let you handle and raise
Python exceptions. It is important to understand some of the basics of
Python exception handling. It works somewhat like the POSIX :c`errno`
variable: there is a global indicator (per thread) of the last error
that occurred. Most C API functions don't clear this on success, but
will set it to indicate the cause of the error on failure. Most C API
functions also return an error indicator, usually `NULL` if they are
supposed to return a pointer, or `-1` if they return an integer
(exception: the :c`PyArg_\*` functions return `1` for success and `0`
for failure).

Concretely, the error indicator consists of three object pointers: the
exception's type, the exception's value, and the traceback object. Any
of those pointers can be `NULL` if non-set (although some combinations
are forbidden, for example you can't have a non-`NULL` traceback if the
exception type is `NULL`).

When a function must fail because some function it called failed, it
generally doesn't set the error indicator; the function it called
already set it. It is responsible for either handling the error and
clearing the exception or returning after cleaning up any resources it
holds (such as object references or memory allocations); it should *not*
continue normally if it is not prepared to handle the error. If
returning due to an error, it is important to indicate to the caller
that an error has been set. If the error is not handled or carefully
propagated, additional calls into the Python/C API may not behave as
intended and may fail in mysterious ways.

Note

The error indicator is **not** the result of `sys.exc_info()`. The
former corresponds to an exception that is not yet caught (and is
therefore still propagating), while the latter returns an exception
after it is caught (and has therefore stopped propagating).

## Printing and clearing

> Clear the error indicator. If the error indicator is not set, there is
> no effect.

> Print a standard traceback to `sys.stderr` and clear the error
> indicator. **Unless** the error is a `SystemExit`, in that case no
> traceback is printed and the Python process will exit with the error
> code specified by the `SystemExit` instance.
>
> Call this function **only** when the error indicator is set. Otherwise
> it will cause a fatal error!
>
> If *set_sys_last_vars* is nonzero, the variables `sys.last_type`,
> `sys.last_value` and `sys.last_traceback` will be set to the type,
> value and traceback of the printed exception, respectively.

> Alias for `PyErr_PrintEx(1)`.

> Call `sys.unraisablehook` using the current exception and *obj*
> argument.
>
> This utility function prints a warning message to `sys.stderr` when an
> exception has been set but it is impossible for the interpreter to
> actually raise the exception. It is used, for example, when an
> exception occurs in an `__del__` method.
>
> The function is called with a single argument *obj* that identifies
> the context in which the unraisable exception occurred. If possible,
> the repr of *obj* will be printed in the warning message.
>
> An exception must be set when calling this function.

## Raising exceptions

These functions help you set the current thread's error indicator. For
convenience, some of these functions will always return a `NULL` pointer
for use in a `return` statement.

> This is the most common way to set the error indicator. The first
> argument specifies the exception type; it is normally one of the
> standard exceptions, e.g. :c`PyExc_RuntimeError`. You need not
> increment its reference count. The second argument is an error
> message; it is decoded from `'utf-8'`.

> This function is similar to :c`PyErr_SetString` but lets you specify
> an arbitrary Python object for the "value" of the exception.

> This function sets the error indicator and returns `NULL`. *exception*
> should be a Python exception class. The *format* and subsequent
> parameters help format the error message; they have the same meaning
> and values as in :c`PyUnicode_FromFormat`. *format* is an
> ASCII-encoded string.

> Same as :c`PyErr_Format`, but taking a :c`va_list` argument rather
> than a variable number of arguments.
>
> 3.5

> This is a shorthand for `PyErr_SetObject(type, Py_None)`.

> This is a shorthand for `PyErr_SetString(PyExc_TypeError, message)`,
> where *message* indicates that a built-in operation was invoked with
> an illegal argument. It is mostly for internal use.

> This is a shorthand for `PyErr_SetNone(PyExc_MemoryError)`; it returns
> `NULL` so an object allocation function can write
> `return PyErr_NoMemory();` when it runs out of memory.

> single: strerror()
>
> This is a convenience function to raise an exception when a C library
> function has returned an error and set the C variable :c`errno`. It
> constructs a tuple object whose first item is the integer :c`errno`
> value and whose second item is the corresponding error message (gotten
> from :c`strerror`), and then calls `PyErr_SetObject(type, object)`. On
> Unix, when the :c`errno` value is `EINTR`, indicating an interrupted
> system call, this calls :c`PyErr_CheckSignals`, and if that set the
> error indicator, leaves it set to that. The function always returns
> `NULL`, so a wrapper function around a system call can write
> `return PyErr_SetFromErrno(type);` when the system call returns an
> error.

> Similar to :c`PyErr_SetFromErrno`, with the additional behavior that
> if *filenameObject* is not `NULL`, it is passed to the constructor of
> *type* as a third parameter. In the case of `OSError` exception, this
> is used to define the `filename` attribute of the exception instance.

> Similar to :c`PyErr_SetFromErrnoWithFilenameObject`, but takes a
> second filename object, for raising errors when a function that takes
> two filenames fails.
>
> 3.4

> Similar to :c`PyErr_SetFromErrnoWithFilenameObject`, but the filename
> is given as a C string. *filename* is decoded from the `filesystem
> encoding and error handler`.

> This is a convenience function to raise `WindowsError`. If called with
> *ierr* of :c`0`, the error code returned by a call to :c`GetLastError`
> is used instead. It calls the Win32 function :c`FormatMessage` to
> retrieve the Windows description of error code given by *ierr* or
> :c`GetLastError`, then it constructs a tuple object whose first item
> is the *ierr* value and whose second item is the corresponding error
> message (gotten from :c`FormatMessage`), and then calls
> `PyErr_SetObject(PyExc_WindowsError, object)`. This function always
> returns `NULL`.
>
> Windows.

> Similar to :c`PyErr_SetFromWindowsErr`, with an additional parameter
> specifying the exception type to be raised.
>
> Windows.

> Similar to :c`PyErr_SetFromWindowsErrWithFilenameObject`, but the
> filename is given as a C string. *filename* is decoded from the
> filesystem encoding (`os.fsdecode`).
>
> Windows.

> Similar to :c`PyErr_SetFromWindowsErrWithFilenameObject`, with an
> additional parameter specifying the exception type to be raised.
>
> Windows.

> Similar to :c`PyErr_SetExcFromWindowsErrWithFilenameObject`, but
> accepts a second filename object.
>
> Windows.
>
> 3.4

> Similar to :c`PyErr_SetFromWindowsErrWithFilename`, with an additional
> parameter specifying the exception type to be raised.
>
> Windows.

> This is a convenience function to raise `ImportError`. *msg* will be
> set as the exception's message string. *name* and *path*, both of
> which can be `NULL`, will be set as the `ImportError`'s respective
> `name` and `path` attributes.
>
> 3.3

> Much like :c`PyErr_SetImportError` but this function allows for
> specifying a subclass of `ImportError` to raise.
>
> 3.6

> Set file, line, and offset information for the current exception. If
> the current exception is not a `SyntaxError`, then it sets additional
> attributes, which make the exception printing subsystem think the
> exception is a `SyntaxError`.
>
> 3.4

> Like :c`PyErr_SyntaxLocationObject`, but *filename* is a byte string
> decoded from the `filesystem encoding and error handler`.
>
> 3.2

> Like :c`PyErr_SyntaxLocationEx`, but the *col_offset* parameter is
> omitted.

> This is a shorthand for `PyErr_SetString(PyExc_SystemError, message)`,
> where *message* indicates that an internal operation (e.g. a Python/C
> API function) was invoked with an illegal argument. It is mostly for
> internal use.

## Issuing warnings

Use these functions to issue warnings from C code. They mirror similar
functions exported by the Python `warnings` module. They normally print
a warning message to *sys.stderr*; however, it is also possible that the
user has specified that warnings are to be turned into errors, and in
that case they will raise an exception. It is also possible that the
functions raise an exception because of a problem with the warning
machinery. The return value is `0` if no exception is raised, or `-1` if
an exception is raised. (It is not possible to determine whether a
warning message is actually printed, nor what the reason is for the
exception; this is intentional.) If an exception is raised, the caller
should do its normal exception handling (for example, :c`Py_DECREF`
owned references and return an error value).

> Issue a warning message. The *category* argument is a warning category
> (see below) or `NULL`; the *message* argument is a UTF-8 encoded
> string. *stack_level* is a positive number giving a number of stack
> frames; the warning will be issued from the currently executing line
> of code in that stack frame. A *stack_level* of 1 is the function
> calling :c`PyErr_WarnEx`, 2 is the function above that, and so forth.
>
> Warning categories must be subclasses of :c`PyExc_Warning`;
> :c`PyExc_Warning` is a subclass of :c`PyExc_Exception`; the default
> warning category is :c`PyExc_RuntimeWarning`. The standard Python
> warning categories are available as global variables whose names are
> enumerated at `standardwarningcategories`.
>
> For information about warning control, see the documentation for the
> `warnings` module and the `-W` option in the command line
> documentation. There is no C API for warning control.

> Issue a warning message with explicit control over all warning
> attributes. This is a straightforward wrapper around the Python
> function `warnings.warn_explicit`; see there for more information. The
> *module* and *registry* arguments may be set to `NULL` to get the
> default effect described there.
>
> 3.4

> Similar to :c`PyErr_WarnExplicitObject` except that *message* and
> *module* are UTF-8 encoded strings, and *filename* is decoded from the
> `filesystem encoding and error handler`.

> Function similar to :c`PyErr_WarnEx`, but use :c`PyUnicode_FromFormat`
> to format the warning message. *format* is an ASCII-encoded string.
>
> 3.2

> Function similar to :c`PyErr_WarnFormat`, but *category* is
> `ResourceWarning` and it passes *source* to `warnings.WarningMessage`.
>
> 3.6

## Querying the error indicator

> Test whether the error indicator is set. If set, return the exception
> *type* (the first argument to the last call to one of the
> :c`PyErr_Set\*` functions or to :c`PyErr_Restore`). If not set, return
> `NULL`. You do not own a reference to the return value, so you do not
> need to :c`Py_DECREF` it.
>
> The caller must hold the GIL.
>
> Note
>
> Do not compare the return value to a specific exception; use
> :c`PyErr_ExceptionMatches` instead, shown below. (The comparison could
> easily fail since the exception may be an instance instead of a class,
> in the case of a class exception, or it may be a subclass of the
> expected exception.)

> Equivalent to `PyErr_GivenExceptionMatches(PyErr_Occurred(), exc)`.
> This should only be called when an exception is actually set; a memory
> access violation will occur if no exception has been raised.

> Return true if the *given* exception matches the exception type in
> *exc*. If *exc* is a class object, this also returns true when *given*
> is an instance of a subclass. If *exc* is a tuple, all exception types
> in the tuple (and recursively in subtuples) are searched for a match.

> Retrieve the error indicator into three variables whose addresses are
> passed. If the error indicator is not set, set all three variables to
> `NULL`. If it is set, it will be cleared and you own a reference to
> each object retrieved. The value and traceback object may be `NULL`
> even when the type object is not.
>
> Note
>
> This function is normally only used by code that needs to catch
> exceptions or by code that needs to save and restore the error
> indicator temporarily, e.g.:
>
> ``` c
> {
>    PyObject *type, *value, *traceback;
>    PyErr_Fetch(&type, &value, &traceback);
>
>    /* ... code that might produce other errors ... */
>
>    PyErr_Restore(type, value, traceback);
> }
> ```

> Set the error indicator from the three objects. If the error indicator
> is already set, it is cleared first. If the objects are `NULL`, the
> error indicator is cleared. Do not pass a `NULL` type and non-`NULL`
> value or traceback. The exception type should be a class. Do not pass
> an invalid exception type or value. (Violating these rules will cause
> subtle problems later.) This call takes away a reference to each
> object: you must own a reference to each object before the call and
> after the call you no longer own these references. (If you don't
> understand this, don't use this function. I warned you.)
>
> Note
>
> This function is normally only used by code that needs to save and
> restore the error indicator temporarily. Use :c`PyErr_Fetch` to save
> the current error indicator.

> Under certain circumstances, the values returned by :c`PyErr_Fetch`
> below can be "unnormalized", meaning that `*exc` is a class object but
> `*val` is not an instance of the same class. This function can be used
> to instantiate the class in that case. If the values are already
> normalized, nothing happens. The delayed normalization is implemented
> to improve performance.
>
> Note
>
> This function *does not* implicitly set the `__traceback__` attribute
> on the exception value. If setting the traceback appropriately is
> desired, the following additional snippet is needed:
>
> ``` c
> if (tb != NULL) {
>   PyException_SetTraceback(val, tb);
> }
> ```

> Retrieve the active exception instance, as would be returned by
> `sys.exception`. This refers to an exception that was *already
> caught*, not to an exception that was freshly raised. Returns a new
> reference to the exception or `NULL`. Does not modify the
> interpreter's exception state.
>
> Note
>
> This function is not normally used by code that wants to handle
> exceptions. Rather, it can be used when code needs to save and restore
> the exception state temporarily. Use :c`PyErr_SetHandledException` to
> restore or clear the exception state.
>
> 3.11

> Set the active exception, as known from `sys.exception()`. This refers
> to an exception that was *already caught*, not to an exception that
> was freshly raised. To clear the exception state, pass `NULL`.
>
> Note
>
> This function is not normally used by code that wants to handle
> exceptions. Rather, it can be used when code needs to save and restore
> the exception state temporarily. Use :c`PyErr_GetHandledException` to
> get the exception state.
>
> 3.11

> Retrieve the old-style representation of the exception info, as known
> from `sys.exc_info`. This refers to an exception that was *already
> caught*, not to an exception that was freshly raised. Returns new
> references for the three objects, any of which may be `NULL`. Does not
> modify the exception info state. This function is kept for backwards
> compatibility. Prefer using :c`PyErr_GetHandledException`.
>
> Note
>
> This function is not normally used by code that wants to handle
> exceptions. Rather, it can be used when code needs to save and restore
> the exception state temporarily. Use :c`PyErr_SetExcInfo` to restore
> or clear the exception state.
>
> 3.3

> Set the exception info, as known from `sys.exc_info()`. This refers to
> an exception that was *already caught*, not to an exception that was
> freshly raised. This function steals the references of the arguments.
> To clear the exception state, pass `NULL` for all three arguments.
> This function is kept for backwards compatibility. Prefer using
> :c`PyErr_SetHandledException`.
>
> Note
>
> This function is not normally used by code that wants to handle
> exceptions. Rather, it can be used when code needs to save and restore
> the exception state temporarily. Use :c`PyErr_GetExcInfo` to read the
> exception state.
>
> 3.3
>
> 3.11 The `type` and `traceback` arguments are no longer used and can
> be NULL. The interpreter now derives them from the exception instance
> (the `value` argument). The function still steals references of all
> three arguments.

## Signal Handling

> module: signal single: SIGINT single: KeyboardInterrupt (built-in
> exception)
>
> This function interacts with Python's signal handling.
>
> If the function is called from the main thread and under the main
> Python interpreter, it checks whether a signal has been sent to the
> processes and if so, invokes the corresponding signal handler. If the
> `signal` module is supported, this can invoke a signal handler written
> in Python.
>
> The function attempts to handle all pending signals, and then returns
> `0`. However, if a Python signal handler raises an exception, the
> error indicator is set and the function returns `-1` immediately (such
> that other pending signals may not have been handled yet: they will be
> on the next :c`PyErr_CheckSignals()` invocation).
>
> If the function is called from a non-main thread, or under a non-main
> Python interpreter, it does nothing and returns `0`.
>
> This function can be called by long-running C code that wants to be
> interruptible by user requests (such as by pressing Ctrl-C).
>
> Note
>
> The default Python signal handler for `SIGINT` raises the
> `KeyboardInterrupt` exception.

> module: signal single: SIGINT single: KeyboardInterrupt (built-in
> exception)
>
> Simulate the effect of a `SIGINT` signal arriving. This is equivalent
> to `PyErr_SetInterruptEx(SIGINT)`.
>
> Note
>
> This function is async-signal-safe. It can be called without the `GIL`
> and from a C signal handler.

> module: signal single: KeyboardInterrupt (built-in exception)
>
> Simulate the effect of a signal arriving. The next time
> :c`PyErr_CheckSignals` is called, the Python signal handler for the
> given signal number will be called.
>
> This function can be called by C code that sets up its own signal
> handling and wants Python signal handlers to be invoked as expected
> when an interruption is requested (for example when the user presses
> Ctrl-C to interrupt an operation).
>
> If the given signal isn't handled by Python (it was set to
> `signal.SIG_DFL` or `signal.SIG_IGN`), it will be ignored.
>
> If *signum* is outside of the allowed range of signal numbers, `-1` is
> returned. Otherwise, `0` is returned. The error indicator is never
> changed by this function.
>
> Note
>
> This function is async-signal-safe. It can be called without the `GIL`
> and from a C signal handler.
>
> 3.10

> This utility function specifies a file descriptor to which the signal
> number is written as a single byte whenever a signal is received. *fd*
> must be non-blocking. It returns the previous such file descriptor.
>
> The value `-1` disables the feature; this is the initial state. This
> is equivalent to `signal.set_wakeup_fd` in Python, but without any
> error checking. *fd* should be a valid file descriptor. The function
> should only be called from the main thread.
>
> 3.5 On Windows, the function now also supports socket handles.

## Exception Classes

> This utility function creates and returns a new exception class. The
> *name* argument must be the name of the new exception, a C string of
> the form `module.classname`. The *base* and *dict* arguments are
> normally `NULL`. This creates a class object derived from `Exception`
> (accessible in C as :c`PyExc_Exception`).
>
> The `__module__` attribute of the new class is set to the first part
> (up to the last dot) of the *name* argument, and the class name is set
> to the last part (after the last dot). The *base* argument can be used
> to specify alternate base classes; it can either be only one class or
> a tuple of classes. The *dict* argument can be used to specify a
> dictionary of class variables and methods.

> Same as :c`PyErr_NewException`, except that the new exception class
> can easily be given a docstring: If *doc* is non-`NULL`, it will be
> used as the docstring for the exception class.
>
> 3.2

## Exception Objects

> Return the traceback associated with the exception as a new reference,
> as accessible from Python through `__traceback__`. If there is no
> traceback associated, this returns `NULL`.

> Set the traceback associated with the exception to *tb*. Use `Py_None`
> to clear it.

> Return the context (another exception instance during whose handling
> *ex* was raised) associated with the exception as a new reference, as
> accessible from Python through `__context__`. If there is no context
> associated, this returns `NULL`.

> Set the context associated with the exception to *ctx*. Use `NULL` to
> clear it. There is no type check to make sure that *ctx* is an
> exception instance. This steals a reference to *ctx*.

> Return the cause (either an exception instance, or `None`, set by
> `raise ... from ...`) associated with the exception as a new
> reference, as accessible from Python through `__cause__`.

> Set the cause associated with the exception to *cause*. Use `NULL` to
> clear it. There is no type check to make sure that *cause* is either
> an exception instance or `None`. This steals a reference to *cause*.
>
> `__suppress_context__` is implicitly set to `True` by this function.

## Unicode Exception Objects

The following functions are used to create and modify Unicode exceptions
from C.

> Create a `UnicodeDecodeError` object with the attributes *encoding*,
> *object*, *length*, *start*, *end* and *reason*. *encoding* and
> *reason* are UTF-8 encoded strings.

## Recursion Control

These two functions provide a way to perform safe recursive calls at the
C level, both in the core and in extension modules. They are needed if
the recursive code does not necessarily invoke Python code (which tracks
its recursion depth automatically). They are also not needed for
*tp_call* implementations because the `call protocol <call>` takes care
of recursion handling.

> Marks a point where a recursive C-level call is about to be performed.
>
> If `USE_STACKCHECK` is defined, this function checks if the OS stack
> overflowed using :c`PyOS_CheckStack`. In this is the case, it sets a
> `MemoryError` and returns a nonzero value.
>
> The function then checks if the recursion limit is reached. If this is
> the case, a `RecursionError` is set and a nonzero value is returned.
> Otherwise, zero is returned.
>
> *where* should be a UTF-8 encoded string such as
> `" in instance check"` to be concatenated to the `RecursionError`
> message caused by the recursion depth limit.
>
> 3.9 This function is now also available in the limited API.

> Ends a :c`Py_EnterRecursiveCall`. Must be called once for each
> *successful* invocation of :c`Py_EnterRecursiveCall`.
>
> 3.9 This function is now also available in the limited API.

Properly implementing :c`~PyTypeObject.tp_repr` for container types
requires special recursion handling. In addition to protecting the
stack, :c`~PyTypeObject.tp_repr` also needs to track objects to prevent
cycles. The following two functions facilitate this functionality.
Effectively, these are the C equivalent to `reprlib.recursive_repr`.

> Called at the beginning of the :c`~PyTypeObject.tp_repr`
> implementation to detect cycles.
>
> If the object has already been processed, the function returns a
> positive integer. In that case the :c`~PyTypeObject.tp_repr`
> implementation should return a string object indicating a cycle. As
> examples, `dict` objects return `{...}` and `list` objects return
> `[...]`.
>
> The function will return a negative integer if the recursion limit is
> reached. In that case the :c`~PyTypeObject.tp_repr` implementation
> should typically return `NULL`.
>
> Otherwise, the function returns zero and the :c`~PyTypeObject.tp_repr`
> implementation can continue normally.

> Ends a :c`Py_ReprEnter`. Must be called once for each invocation of
> :c`Py_ReprEnter` that returns zero.

## Standard Exceptions

All standard Python exceptions are available as global variables whose
names are `PyExc_` followed by the Python exception name. These have the
type :c`PyObject*`; they are all class objects. For completeness, here
are all the variables:

single: PyExc_BaseException single: PyExc_Exception single:
PyExc_ArithmeticError single: PyExc_AssertionError single:
PyExc_AttributeError single: PyExc_BlockingIOError single:
PyExc_BrokenPipeError single: PyExc_BufferError single:
PyExc_ChildProcessError single: PyExc_ConnectionAbortedError single:
PyExc_ConnectionError single: PyExc_ConnectionRefusedError single:
PyExc_ConnectionResetError single: PyExc_EOFError single:
PyExc_FileExistsError single: PyExc_FileNotFoundError single:
PyExc_FloatingPointError single: PyExc_GeneratorExit single:
PyExc_ImportError single: PyExc_IndentationError single:
PyExc_IndexError single: PyExc_InterruptedError single:
PyExc_IsADirectoryError single: PyExc_KeyError single:
PyExc_KeyboardInterrupt single: PyExc_LookupError single:
PyExc_MemoryError single: PyExc_ModuleNotFoundError single:
PyExc_NameError single: PyExc_NotADirectoryError single:
PyExc_NotImplementedError single: PyExc_OSError single:
PyExc_OverflowError single: PyExc_PermissionError single:
PyExc_ProcessLookupError single: PyExc_RecursionError single:
PyExc_ReferenceError single: PyExc_RuntimeError single:
PyExc_StopAsyncIteration single: PyExc_StopIteration single:
PyExc_SyntaxError single: PyExc_SystemError single: PyExc_SystemExit
single: PyExc_TabError single: PyExc_TimeoutError single:
PyExc_TypeError single: PyExc_UnboundLocalError single:
PyExc_UnicodeDecodeError single: PyExc_UnicodeEncodeError single:
PyExc_UnicodeError single: PyExc_UnicodeTranslateError single:
PyExc_ValueError single: PyExc_ZeroDivisionError

| C Name                           | Python Name              | Notes |
|----------------------------------|--------------------------|-------|
| :c`PyExc_BaseException`          | `BaseException`          | [1]   |
| :c`PyExc_Exception`              | `Exception`              | [2]   |
| :c`PyExc_ArithmeticError`        | `ArithmeticError`        | [3]   |
| :c`PyExc_AssertionError`         | `AssertionError`         |       |
| :c`PyExc_AttributeError`         | `AttributeError`         |       |
| :c`PyExc_BlockingIOError`        | `BlockingIOError`        |       |
| :c`PyExc_BrokenPipeError`        | `BrokenPipeError`        |       |
| :c`PyExc_BufferError`            | `BufferError`            |       |
| :c`PyExc_ChildProcessError`      | `ChildProcessError`      |       |
| :c`PyExc_ConnectionAbortedError` | `ConnectionAbortedError` |       |
| :c`PyExc_ConnectionError`        | `ConnectionError`        |       |
| :c`PyExc_ConnectionRefusedError` | `ConnectionRefusedError` |       |
| :c`PyExc_ConnectionResetError`   | `ConnectionResetError`   |       |
| :c`PyExc_EOFError`               | `EOFError`               |       |
| :c`PyExc_FileExistsError`        | `FileExistsError`        |       |
| :c`PyExc_FileNotFoundError`      | `FileNotFoundError`      |       |
| :c`PyExc_FloatingPointError`     | `FloatingPointError`     |       |
| :c`PyExc_GeneratorExit`          | `GeneratorExit`          |       |
| :c`PyExc_ImportError`            | `ImportError`            |       |
| :c`PyExc_IndentationError`       | `IndentationError`       |       |
| :c`PyExc_IndexError`             | `IndexError`             |       |
| :c`PyExc_InterruptedError`       | `InterruptedError`       |       |
| :c`PyExc_IsADirectoryError`      | `IsADirectoryError`      |       |
| :c`PyExc_KeyError`               | `KeyError`               |       |
| :c`PyExc_KeyboardInterrupt`      | `KeyboardInterrupt`      |       |
| :c`PyExc_LookupError`            | `LookupError`            | [4]   |
| :c`PyExc_MemoryError`            | `MemoryError`            |       |
| :c`PyExc_ModuleNotFoundError`    | `ModuleNotFoundError`    |       |
| :c`PyExc_NameError`              | `NameError`              |       |
| :c`PyExc_NotADirectoryError`     | `NotADirectoryError`     |       |
| :c`PyExc_NotImplementedError`    | `NotImplementedError`    |       |
| :c`PyExc_OSError`                | `OSError`                | [5]   |
| :c`PyExc_OverflowError`          | `OverflowError`          |       |
| :c`PyExc_PermissionError`        | `PermissionError`        |       |
| :c`PyExc_ProcessLookupError`     | `ProcessLookupError`     |       |
| :c`PyExc_RecursionError`         | `RecursionError`         |       |
| :c`PyExc_ReferenceError`         | `ReferenceError`         |       |
| :c`PyExc_RuntimeError`           | `RuntimeError`           |       |
| :c`PyExc_StopAsyncIteration`     | `StopAsyncIteration`     |       |
| :c`PyExc_StopIteration`          | `StopIteration`          |       |
| :c`PyExc_SyntaxError`            | `SyntaxError`            |       |
| :c`PyExc_SystemError`            | `SystemError`            |       |
| :c`PyExc_SystemExit`             | `SystemExit`             |       |
| :c`PyExc_TabError`               | `TabError`               |       |
| :c`PyExc_TimeoutError`           | `TimeoutError`           |       |
| :c`PyExc_TypeError`              | `TypeError`              |       |
| :c`PyExc_UnboundLocalError`      | `UnboundLocalError`      |       |
| :c`PyExc_UnicodeDecodeError`     | `UnicodeDecodeError`     |       |
| :c`PyExc_UnicodeEncodeError`     | `UnicodeEncodeError`     |       |
| :c`PyExc_UnicodeError`           | `UnicodeError`           |       |
| :c`PyExc_UnicodeTranslateError`  | `UnicodeTranslateError`  |       |
| :c`PyExc_ValueError`             | `ValueError`             |       |
| :c`PyExc_ZeroDivisionError`      | `ZeroDivisionError`      |       |

3.3 :c`PyExc_BlockingIOError`, :c`PyExc_BrokenPipeError`,
:c`PyExc_ChildProcessError`, :c`PyExc_ConnectionError`,
:c`PyExc_ConnectionAbortedError`, :c`PyExc_ConnectionRefusedError`,
:c`PyExc_ConnectionResetError`, :c`PyExc_FileExistsError`,
:c`PyExc_FileNotFoundError`, :c`PyExc_InterruptedError`,
:c`PyExc_IsADirectoryError`, :c`PyExc_NotADirectoryError`,
:c`PyExc_PermissionError`, :c`PyExc_ProcessLookupError` and
:c`PyExc_TimeoutError` were introduced following `3151`.

3.5 :c`PyExc_StopAsyncIteration` and :c`PyExc_RecursionError`.

3.6 :c`PyExc_ModuleNotFoundError`.

These are compatibility aliases to :c`PyExc_OSError`:

single: PyExc_EnvironmentError single: PyExc_IOError single:
PyExc_WindowsError

| C Name                     | Notes |
|----------------------------|-------|
| :c`PyExc_EnvironmentError` |       |
| :c`PyExc_IOError`          |       |
| :c`PyExc_WindowsError`     | [6]   |

3.3 These aliases used to be separate exception types.

Notes:

## Standard Warning Categories

All standard Python warning categories are available as global variables
whose names are `PyExc_` followed by the Python exception name. These
have the type :c`PyObject*`; they are all class objects. For
completeness, here are all the variables:

single: PyExc_Warning single: PyExc_BytesWarning single:
PyExc_DeprecationWarning single: PyExc_FutureWarning single:
PyExc_ImportWarning single: PyExc_PendingDeprecationWarning single:
PyExc_ResourceWarning single: PyExc_RuntimeWarning single:
PyExc_SyntaxWarning single: PyExc_UnicodeWarning single:
PyExc_UserWarning

| C Name                              | Python Name                 | Notes |
|-------------------------------------|-----------------------------|-------|
| :c`PyExc_Warning`                   | `Warning`                   | [7]   |
| :c`PyExc_BytesWarning`              | `BytesWarning`              |       |
| :c`PyExc_DeprecationWarning`        | `DeprecationWarning`        |       |
| :c`PyExc_FutureWarning`             | `FutureWarning`             |       |
| :c`PyExc_ImportWarning`             | `ImportWarning`             |       |
| :c`PyExc_PendingDeprecationWarning` | `PendingDeprecationWarning` |       |
| :c`PyExc_ResourceWarning`           | `ResourceWarning`           |       |
| :c`PyExc_RuntimeWarning`            | `RuntimeWarning`            |       |
| :c`PyExc_SyntaxWarning`             | `SyntaxWarning`             |       |
| :c`PyExc_UnicodeWarning`            | `UnicodeWarning`            |       |
| :c`PyExc_UserWarning`               | `UserWarning`               |       |

3.2 :c`PyExc_ResourceWarning`.

Notes:

[1] This is a base class for other standard exceptions.

[2] This is a base class for other standard exceptions.

[3] This is a base class for other standard exceptions.

[4] This is a base class for other standard exceptions.

[5] This is a base class for other standard exceptions.

[6] Only defined on Windows; protect code that uses this by testing that
the preprocessor macro `MS_WINDOWS` is defined.

[7] This is a base class for other standard warning categories.